[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ishandandekar/What_Am_I_Eating/blob/main/notebooks/make_the_batter.ipynb)

# What_Am_I_Eating
👋Hello and welcome to the `bake_the_batter` notebook. This is the notebook used to create the models. I'll use methods such as transfer learning to get the `EfficientNetB0` model for the neural network. I'll get the data from tensorflow-datasets. If you want to replicate this project, you can use this notebook to make the models.

### Setting up the environment
We'll need a GPU (which our kind Google will provide us), this will be required for tensorflow and mixed precision. We'll also need some helper functions to analyse the models.

In [ ]:
# Importing tensorflow and some other libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

!nvidia-smi -L

In [ ]:
# Downloading the helper functions
!wget https://raw.githubusercontent.com/ishandandekar/What_Am_I_Eating/main/helper_functions.py

from helper_functions import plot_loss_curves, show_random_samples

### Getting the data
I used the Food 101 data to train the models. Although the dataset is available on [Kaggle](https://www.kaggle.com/datasets/dansbecker/food-101), I'll use it from tensorflow-datasets.  

What is **TensorFlow Datasets**?
A place for prepared and ready-to-use machine learning datasets.

Why use TensorFlow Datasets?
* Load data already in Tensors
* Practice on well established datasets

Why not use TensorFlow Datasets?
* The datasets are static (they don't change, like your real-world datasets would)
* Might not be suited for your particular problem (but great for experimenting)

In [ ]:
# Get the tensorflow-datasets library
import tensorflow_datasets as tfds

# Checking if Food 101 is in tensorflow-datasets catalog
dataset_list = tfds.list_builders()
print(f"Is Food 101 in the dataset catalog: {'food101' in dataset_list}")

Great! Now let's get the dataset. Tensorflow is kind to us, it already has the train and validation sets of the data. Hence, we don't need to split the data. It is a large dataset around 5 GBs of images. Please be careful while downloading the dataset.  
> 💡**Note:** You can also add an additional argument in the `load` function to specify the directory. Check the documentation to know more. https://www.tensorflow.org/datasets/api_docs/python/tfds/load

In [ ]:
# Load in tha data (takes 5-6 minutes in Google Colab)
(train_data,test_data),ds_info = tfds.load(name="food101",
                                           split=["train","validation"],
                                           shuffle_files=True,
                                           as_supervised=True, # data gets returned in tuple format (data,label)
                                           with_info=True)

# Information about the data
print(ds_info)

### Exploring the data  
In this section, we'll go through the data, check if there are any discrepancies i.e. is the data labelled wrong? We'll also have to check the dimensions and shape of the data and make sure it is safe for building our neural network on top of it.  
Let's get the class names. There 101 classes in the dataset. We'll see any 5 of those, to have a look at what we are dealing with.

In [ ]:
class_names = class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# Get random data images from the directory
show_random_samples(dir_name=train_data, class_names=class_names)

Our images have random shape and unwanted datatype. Tensorflow works well with float datatype and our images have `uint8`. We'll have to make a function for preprocessing this data and converting to our desired format.

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image,label,img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32'
  image to [img_shape,img_shape,color_channels]
  """
  image = tf.image.resize(image,[img_shape,img_shape])
  return tf.cast(image,tf.float32), label

Now that we have a function to process our data. Let's use the `map` function on our train and test data to format our images.

In [ ]:
# Map preprocessing function onto our training set
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch it
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

> 💡**Note:** These functions we used, `batch` and `prefetch` will help us when we set the model to train. These functions will help us train the model faster. It optimizes the space and complexity immensely.

### Setup mixed precision for training
First and foremost, for a deeper understanding of mized precision training, check out [Tensorflow guide](https://www.tensorflow.org/guide/mixed_precision) for mixed precision.  

Mixed precision utilizes a combination of float32 and float16 data types to speed up model performance

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

The model in the deployed web app does not use mixed precision!

### Building the model
This is where the best part comes in. Although it is fairly easy to make CNNs and fit the model, we'll use the `EfficientNetB0` model which is pre-trained on the [ImageNet](https://image-net.org/) dataset.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import RandomZoom,RandomFlip,RandomRotation,RandomHeight,RandomWidth
from tensorflow.keras import Sequential

# Create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# This will help us to fine tune the model on our train set
base_model.trainable=True

# Setup data augmentation
data_augmentation = Sequential(
    [
     RandomFlip("horizontal"),
     RandomRotation(0.2),
     RandomHeight(0.2),
     RandomWidth(0.2),
     RandomZoom(0.2),
    #  layers.Rescaling(1/255.0) # Resnet needs rescaling!
    ],name="data_augmentation"
)

# Create a functional model
inputs = layers.Input(shape=input_shape, name="Input_layer")

# Note: EfficientNetBX models have rescaling built-in
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax",dtype=tf.float32,name="softmax_float32")(x) # If using mixed precision add argument dtype=tf.float32
model = tf.keras.Model(inputs,outputs)

In [ ]:
# Setting up callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3,restore_best_weights=True)

checkpoint_path = "model_checkpoint/fntd.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,monitor="val_accuracy",save_best_only=True,save_weights_only=True,verbose=1)

Maybe you can also add a learning rate callback too!

In [ ]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
# Fit the model
history_fine_tuned_all = model.fit(train_data,
                                   epochs=10,
                                   steps_per_epoch=len(train_data),
                                   validation_data=test_data,
                                   validation_steps=int(0.15*len(test_data)),
                                   callbacks=[model_checkpoint_callback,early_stopping_callback])

> 💡**Note:** We'll only evaluate on 15% of our validation data, as it'll speed up the training process.

### Evaluating the model

In [ ]:
# Evaluating the model on the whole test set
fine_tuned_result = model.evaluate(test_data)
fine_tuned_result

In [ ]:
# Plotting the loss and accuracy of our model
plot_loss_curves(history_fine_tuned_all)

### Save the model

In [ ]:
# Uncomment and run the cell
# model.save("what_am_i_eating_model.h5")